In [ ]:
from loica import *
import matplotlib.pyplot as plt
import numpy as np
import getpass

In [ ]:
from flapjack import *
#fj = Flapjack(url_base='flapjack.rudge-lab.org:8000')
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username=input('Flapjack username: '), password=getpass.getpass('Password: '))

In [ ]:
study = fj.create('study', name='Loica testing', description='Test study for demonstrating Loica')

In [ ]:
dna = fj.create('dna', name='receiver1')
vector = fj.create('vector', name='receiver1', dnas=dna.id)

In [ ]:
network = GeneticNetwork(vector=vector.id[0])

In [ ]:
sfp = fj.create('signal', name='SFP', color='green', description='Simulated fluorescent protein')
reporter = Reporter(name='SFP', color='green', degradation_rate=0, init_concentration=0, signal_id=sfp.id[0])

In [ ]:
network.add_reporter(reporter)

The receiver operator responds to a signal $s$ to produce an output expression rate $\phi(s)$ modeled as follows:

\begin{equation}
    \phi(s)
    =
    \frac
    {
        a + b (\frac{s}{K})^n
    }
    {
        1 + (\frac{s}{K})^n
    }
\end{equation}

In [ ]:
ahl = Supplement(name='AHL1')
rec = Receiver(input=ahl, output=reporter, a=0, b=100, K=1, n=2)
network.add_operator(rec)

In [ ]:
plt.figure(figsize=(3,3), dpi=150)
network.draw()

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.05, 1, 1, 1)

def biomass(t):
    return gompertz(t, 0.05, 1, 1, 1)
    
metab = SimulatedMetabolism(biomass, growth_rate)

media = fj.create('media', name='loica', description='Simulated loica media')
strain = fj.create('strain', name='loica', description='Loica test strain')

In [ ]:
# Create list of samples    
samples = []
concs = np.append(0, np.logspace(-3, 3, 12))
for conc in concs:
    sample = Sample(genetic_network=network, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
    # Add AHL to samples at given concentration
    sample.add_supplement(ahl, conc)
    samples.append(sample)

In [ ]:
biomass_signal = fj.create('signal', name='SOD', description='Simulated OD', color='black')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica receiver1',
              description='Simulated receiver generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run(nsr=0)

In [ ]:
assay.upload(fj, study.id[0])

In [ ]:
ahl1_id = fj.get('chemical', name='AHL1').id[0]
fig = fj.plot(study=study.id,                     
                vector=vector.id,
                signal=sfp.id,
                type='Induction Curve',
                analyte=ahl1_id,
                function='Mean Expression',
                biomass_signal=biomass_signal.id[0],
                normalize='None',
                subplots='Signal',
                markers='Vector',
                plot='All data points'
       )
fig

In [ ]:
rec.characterize(
    fj,
    vector=vector.id,
    media=media.id,
    strain=strain.id,
    signal=sfp.id,
    biomass_signal=biomass_signal.id
)

In [ ]:
rec.a, rec.b, rec.K, rec.n